In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign

In [2]:
# date0='0709_1157'
# date1='0709_1557'
# date2='0709_1934'
# dates=[date0,date1,date2]
# plate = 13
date0='0701_1957'
date1='0701_2357'
date2='0702_0357'
date3='0702_0757'
date4='0702_1157'
date5='0702_1557'
plate=13
dates=[date0,date1,date2,date3,date4,date5]

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [4]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [5]:
nx_graphs=[]
poss=[]
for tab in tabs_labeled:
    nx_graph,pos=generate_nx_graph(tab,labeled=True)
    nx_graphs.append(nx_graph)
    poss.append(pos)

In [6]:
nx_graph_clean=[]
for graph in nx_graphs:
    S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    len_connected=[len(nx_graph.nodes) for nx_graph in S]
    nx_graph_clean.append(S[np.argmax(len_connected)])

In [7]:
skeletons=[]
for nx_graph in nx_graph_clean:
    skeletons.append(generate_skeleton(nx_graph,dim=(20687, 45554)))

In [23]:
xbegin=18000
ybegin=8000
xend=22000
yend=13000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])

In [24]:
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [25]:
plot_t_tp1(node_smalls[1],node_smalls[2],poss[1],poss[2],skeletons_small[1].todense(),skeletons_small[2].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
def compress_skeleton(skeleton_doc,factor):
    shape=skeleton_doc.shape
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for pixel in skeleton_doc.keys():
        x=min(round(pixel[0]/factor),shape[0]//factor-1)
        y=min(round(pixel[1]/factor),shape[1]//factor-1)
        final_picture[x,y]+=1
    return(final_picture)

In [57]:
final_picture1 = compress_skeleton(skeletons[3],5)
final_picture2 = compress_skeleton(skeletons[4],5)

In [54]:
lists_node = [[node for node in nx_graph_aligned.nodes if nx_graph_aligned.degree[node]==1] for nx_graph_aligned in nx_graph_clean]

In [34]:
skeleton_docs_aligned = [sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date}_{plate}_full.npz')) for date in dates]

In [49]:
skeleton_docs_aligned[0]!=skeleton_docs_aligned[1]

<20687x45556 sparse matrix of type '<class 'numpy.bool_'>'
	with 0 stored elements in Compressed Sparse Row format>

In [151]:
final_picture1 = compress_skeleton(skeletons[3],5)
final_picture2 = compress_skeleton(skeletons[4],5)

In [158]:
def pinpoint_anastomosis(nx_graph_tm1,nx_grapht,equ):
    anastomosis=[]
    origins=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    for node in nx_grapht.nodes:
        if nx_grapht.degree(node)>=3 and node in equ.keys() and equ[node] in tips:
            anastomosis.append(node)
            origins.append(equ[node])
    return(anastomosis,origins)

In [133]:
anastomosiss=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i]) for i in range (len(dates)-1)]

In [144]:
new_graph1,new_graph2,new_pos1,new_pos2 = reduce_labels(nx_graph_clean[3],nx_graph_clean[4],poss[3],poss[4])

In [146]:
new_connection = whole_movement_identification(new_graph1,new_graph2,new_pos1,new_pos2)

0.0
0.3003003003003003
0.6006006006006006
0.9009009009009009


In [159]:
anastomosis,origin=pinpoint_anastomosis(new_graph1,new_graph2, new_connection[0])

In [ ]:
plot_t_tp1(origin,anastomosis,new_pos1,new_pos2,final_picture1,final_picture2,compress=5,relabel_t=lambda node : new_connection[0][node])

In [164]:
final_picture1 = compress_skeleton(skeletons[3],5)
final_picture2 = compress_skeleton(skeletons[4],5)
plot_t_tp1(origin,anastomosis,new_pos1,new_pos2,final_picture1,final_picture2,compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [162]:
new_connection[0][950]

517

In [ ]:
plot_t_tp1([],anastomosiss[3],poss[3],poss[4],final_picture1,final_picture2,compress=5)

In [61]:
tip_origin_t0m2_t0m1,pixels_from_tip= whole_movement_identification(nx_graph_clean[3],nx_graph_clean[4],poss[3],poss[4])

0.0
0.3003003003003003
0.6006006006006006
0.9009009009009009


In [106]:
connections_growth_pattern = [whole_movement_identification(nx_graph_clean[i],nx_graph_clean[i+1],poss[i],poss[i+1]) for i in range(len(dates)-1)]

0.0
0.3875968992248062
0.7751937984496124
0.0
0.3831417624521073
0.7662835249042146
0.0
0.33222591362126247
0.6644518272425249
0.9966777408637874
0.0
0.3003003003003003
0.6006006006006006
0.9009009009009009
0.0
0.27472527472527475
0.5494505494505495
0.8241758241758241


In [111]:
connections = [c[0] for c in connections_growth_pattern]
growth_pattern = [c[1] for c in connections_growth_pattern]

In [169]:
growing_tips=[[node for node in growth_pattern[i].keys() if len(growth_pattern[i][node])>=10] for i in range(len(growth_pattern))]

In [180]:
final_picture1 = compress_skeleton(skeletons[3],5)
final_picture2 = compress_skeleton(skeletons[4],5)
plot_t_tp1([],[144],poss[3],poss[4],final_picture1,final_picture2,compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
growth_pattern[3][144]

In [63]:
growths=[len(pixels) for pixels in pixels_from_tip.values()]

In [223]:
node_problem=4607
pos_problem=poss_aligned[0][node_problem]
xbegin=pos_problem[0]-1000
ybegin=pos_problem[1]-1000
xend=pos_problem[0]+1000
yend=pos_problem[1]+1000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graphs_aligned):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss_aligned[i][node][0]<xend and ybegin<poss_aligned[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [224]:
plot_t_tp1([node_problem],[node_problem],poss_aligned[0],poss_aligned[1],skeletons_small[0].todense(),skeletons_small[1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [104]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(np.log(np.array(growths)+1),10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([109.,  37.,  51.,  48.,  14.,  14.,  35.,  13.,   9.,   3.]),
 array([0.        , 0.84031282, 1.68062565, 2.52093847, 3.36125129,
        4.20156412, 5.04187694, 5.88218976, 6.72250259, 7.56281541,
        8.40312824]),
 <BarContainer object of 10 artists>)

In [65]:
def make_growth_picture(pixels_from_tip,pos,shape=(20700,45600),factor=10):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        beginx=round(pos[tip][0]/factor-factor//2)
        endx=round(pos[tip][0]/factor+factor//2)
        beginy=round(pos[tip][1]/factor-factor//2)
        endy=round(pos[tip][1]/factor+factor//2)
        if growth<=200:
            final_picture[beginx:endx,beginy:endy]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    return(final_picture)

In [66]:
def make_growth_picture_per_tip(pixels_from_tip,pos,shape=(20700,45600),factor=10,max_growth=200,min_growth=10,per_tip=True):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    number_tips = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        x=min(round(pos[tip][0]/factor),shape[0]//factor-1)
        y=min(round(pos[tip][1]/factor),shape[1]//factor-1)
        if growth<=max_growth:
#             print(number_tips)
            if growth>=min_growth:
                number_tips[x,y]+=1
                final_picture[x,y]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    if per_tip:
        return(final_picture/(number_tips+(number_tips==0).astype(np.int)),number_tips)
    else:
        return(final_picture,number_tips)

In [92]:
final_picture,number_tips = make_growth_picture_per_tip(pixels_from_tip,poss[3],factor=500,max_growth=4000,per_tip=True,min_growth=0)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(np.log(final_picture+1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:
final_picture1 = compress_skeleton(skeletons[3],500)
final_picture2 = compress_skeleton(skeletons[4],500)

In [100]:
plot_t_tp1([],[],poss[1],poss[2],final_picture,final_picture1,compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
pos_problem=(26*500,48*500)
xbegin=pos_problem[0]-1000
ybegin=pos_problem[1]-1000
xend=pos_problem[0]+1000
yend=pos_problem[1]+1000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [96]:
plot_t_tp1(node_smalls[3],node_smalls[4],poss[3],poss[4],skeletons_small[3].todense(),skeletons_small[4].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
def from_nx_to_tab(nx_graph,pos,tip_origin_corresp,pixel_from_tip):
    column_names = ["origin_label","end_label","origin_pos", "end_pos", "pixel_list"]
    tab = pd.DataFrame(columns=column_names)
    for edge in nx_graph.edges:
        origin_label=edge[0]
        end_label=edge[1]
        origin_pos=pos[origin_label]
        end_pos = pos[end_label]
        pixel_list=orient(nx_graph.get_edge_data(*edge)['pixel_list'],origin_pos)
        new_line=pd.DataFrame({"origin_label":[origin_label],"end_label" : [end_label], "origin_pos":[origin_pos], "end_pos" : [end_pos], "pixel_list" : [pixel_list]})
        tab=tab.append(new_line,ignore_index=True)
    return(tab)

In [388]:
26*500,14*500

(13000, 7000)

In [389]:
pos_problem=(13000, 7000)
xbegin=pos_problem[0]-2000
ybegin=pos_problem[1]-2000
xend=pos_problem[0]+2000
yend=pos_problem[1]+2000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graphs_aligned):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss_aligned[i][node][0]<xend and ybegin<poss_aligned[i][node][1]<yend and nx_graph.degree(node)==1)])

In [390]:
plot_t_tp1(node_smalls[0],node_smalls[1],poss_aligned[0],poss_aligned[1],skeletons_small[0].todense(),skeletons_small[1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:
poss_aligned[1][5136]

array([19343, 33314])

In [315]:
len(pixels_from_tip[27])

197